# Прогнозируем задержки самолетов

In [ ]:
!pip install catboost lightgbm optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
import pandas as pd

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
RANDOM_STATE = 111
DATASET_PATH = 'https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/flight_delays_train.csv'

In [ ]:
data = pd.read_csv(DATASET_PATH)

X = data.drop('dep_delayed_15min', axis=1)
y = data['dep_delayed_15min'] == 'Y'

X.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732
1,c-4,c-20,c-3,1548,US,PIT,MCO,834
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423


Создайте список номеров колонок с категориальными признаками для бустингов

## Quiz
Какой длины получился список?

(подсказка: колонка `DepTime` числовая)

In [ ]:
cat_features = X.select_dtypes(include=['object']).columns.to_list()

Разобъем данные на обучение и контроль

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

In [ ]:
Xtrain.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
41207,c-4,c-18,c-1,1457,CO,EWR,TPA,998
28283,c-11,c-1,c-2,1225,UA,DEN,BOS,1754
34619,c-6,c-16,c-5,1650,YV,IAD,CAE,401
8789,c-5,c-18,c-4,923,AA,SLC,DFW,988
38315,c-2,c-14,c-2,1839,AA,STL,SAN,1558


## Модели с параметрами по умолчанию

Обучите CatBoost с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [ ]:
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import roc_auc_score
model = CatBoostClassifier(random_seed=RANDOM_STATE, verbose=0)
model.fit(Xtrain, ytrain, cat_features=cat_features)
y_pred=model.predict(Xtest)
y_pred_prob=model.predict_proba(Xtest)
roc_auc=roc_auc_score(ytest, y_pred_prob[:,1])
print(round(roc_auc, 2))

0.77


Обучите LightGBM с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [ ]:
for c in X.columns:
    col_type = X[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Xtrain[c] = Xtrain[c].astype('category')
        Xtest[c] = Xtest[c].astype('category')

In [ ]:
model=LGBMClassifier(random_state=RANDOM_STATE)
model.fit(Xtrain, ytrain)

y_pred=model.predict(Xtest)
y_pred_prob=model.predict_proba(Xtest)
roc_auc=roc_auc_score(ytest, y_pred_prob[:,1])
print(round(roc_auc, 2))

[LightGBM] [Info] Number of positive: 14346, number of negative: 60654
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012819 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 75000, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.191280 -> initscore=-1.441714
[LightGBM] [Info] Start training from score -1.441714
0.73


## Optuna

Выделим дополнительную валидационную выборку.

In [ ]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, test_size=0.25, random_state=RANDOM_STATE)

Создайте функцию objective_lgbm, в которой среди гиперпараметров

* num_leaves = trial.suggest_int("num_leaves", 10, 100)
* n_estimators = trial.suggest_int("n_estimators", 10, 1000)

подберите оптимальные, обучая LGBM на Xtrain_new, ytrain_new и проверяя качество (ROC-AUC) на Xval.

Используйте 30 эпох обучения Optuna.


In [ ]:
import optuna

def objective_lgbm(trial):
    # num_leaves (int, optional (default=31)) – Maximum tree leaves for base learners.
    num_leaves = trial.suggest_int("num_leaves", 10, 100)
    # n_estimators (int, optional (default=100)) – Number of boosted trees to fit.
    n_estimators = trial.suggest_int("n_estimators", 10, 1000)

    model=LGBMClassifier(num_leaves=num_leaves, n_estimators=n_estimators, random_state=RANDOM_STATE)
    model.fit(Xtrain_new, ytrain_new)

    # y_pred=model.predict(Xval)
    y_pred_prob=model.predict_proba(Xval)
    roc_auc=roc_auc_score(yval, y_pred_prob[:,1])
    return roc_auc

# storage = optuna.storages.InMemoryStorage()
study = optuna.create_study(direction='maximize')

# Используйте 30 эпох обучения Optuna.
study.optimize(objective_lgbm, n_trials=30)
# {'num_leaves': 65, 'n_estimators': 41}
# {'num_leaves': 52, 'n_estimators': 40}
# {'num_leaves': 52, 'n_estimators': 37}
# {'num_leaves': 96, 'n_estimators': 36}

# study.optimize(objective, n_trials=200)
# {'num_leaves': 13, 'n_estimators': 73}

study.best_params

[I 2024-07-29 08:51:33,448] A new study created in memory with name: no-name-43b9a5cf-8074-4839-aed3-ee0d7699c1d5


[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005286 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-07-29 08:51:43,151] Trial 0 finished with value: 0.7177544320880956 and parameters: {'num_leaves': 65, 'n_estimators': 524}. Best is trial 0 with value: 0.7177544320880956.


[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001614 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-07-29 08:51:47,065] Trial 1 finished with value: 0.720233740092928 and parameters: {'num_leaves': 70, 'n_estimators': 282}. Best is trial 1 with value: 0.720233740092928.


...

[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001444 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-07-29 08:53:13,513] Trial 28 finished with value: 0.7238824868051892 and parameters: {'num_leaves': 31, 'n_estimators': 321}. Best is trial 20 with value: 0.7268737771483303.


[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001414 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-07-29 08:53:17,353] Trial 29 finished with value: 0.7220687485308238 and parameters: {'num_leaves': 18, 'n_estimators': 469}. Best is trial 20 with value: 0.7268737771483303.


{'num_leaves': 10, 'n_estimators': 277}

In [ ]:
study.best_params

Обучите модель с найденными гиперпараметрами на Xtrain, ytrain и оцените ROC-AUC на тестовых данных.

In [ ]:
# your code here

## Quiz

Чему равно количество деревьев в LGBM после подбора гиперпараметров?

## Работа над улучшением модели

* Попробуйте при помощи Optuna подобрать и другие гиперпарамтеры
* Также подберите гиперпараметры у CatBoost (а не только у LightGBM)

In [ ]:
# your code here

## Quiz

Поделитесь своими результатами!